In [4]:
动态规划、DP、水题、搜索、排序、树

['>p(0,1000,-1,"A + B Problem",279997,938154)',
 'p(1,1001,-1,"Sum Problem",169667,672691)',
 'p(0,1002,-1,"A + B Problem II",97595,508271)',
 'p(1,1003,-1,"Max Sum",81529,342922)',
 'p(0,1004,-1,"Let the Balloon Rise",68984,173189)',
 'p(1,1005,-1,"Number Sequence",59147,233017)',
 'p(0,1006,-1,"Tick and Tick",7114,26078)',
 'p(1,1007,-1,"Quoit Design",19947,74820)',
 'p(0,1008,-1,"Elevator",54039,99657)',
 'p(1,1009,-1,"FatMouse\' Trade",39773,115346)',
 'p(0,1010,-1,"Tempter of the Bone",45376,170925)',
 'p(1,1011,-1,"Starship Troopers",6945,30022)',
 'p(0,1012,-1,"u Calculate e",28138,61505)',
 'p(1,1013,-1,"Digital Roots",32619,105288)',
 'p(0,1014,-1,"Uniform Generator",15353,38931)',
 'p(1,1015,-1,"Safecracker",10865,20875)',
 'p(0,1016,-1,"Prime Ring Problem",33468,81141)',
 'p(1,1017,-1,"A Mathematical Curiosity",17981,68276)',
 'p(0,1018,-1,"Big Number",24152,49352)',
 'p(1,1019,-1,"Least Common Multiple",27400,71661)',
 'p(0,1020,-1,"Encoding",28357,63242)',
 'p(1,1021,-1,"F

In [1]:
import re
import csv
import time
import json
import pymysql
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from requests.exceptions import RequestException

def parse_Problem_Archive(text):
    soup = BeautifulSoup(text, 'lxml')
    items = soup.find_all(align='center', height='22')
    for i in range(1, len(items)):
        Pro_ID = items[i].td.text #items[i]中第一个td的文本内容
        tags = set(parse_discuss_page(Pro_ID)) #利用set去重
        tags = ','.join(tags)
        if len(tags) == 0:
            continue
        data = {
            'Pro_ID': Pro_ID,
            'Title': items[i].a.text.strip(),
            'Tags': tags, 
            'Ratio': items[i].td.next_sibling.next_sibling.text[:6],
            'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=' + Pro_ID
        }
        yield data
        
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) '
            + 'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return 'status_code = ' + str(response.status_code)
    except RequestException:
        return 'RequestException'

def parse_discuss_page(Pro_ID):
    url = 'http://acm.hdu.edu.cn/discuss/problem/list.php?problemid=' + str(Pro_ID)
    text = get_one_page(url)
    soup = BeautifulSoup(text, 'lxml')
    discuss_list = soup.select('#ulframe > li > a:nth-child(2)')
    discuss_text = ''
    for d in discuss_list:
        discuss_text += d.text
    match = {
        'dp': re.compile('dp', re.I), #re.I使匹配对大小写不敏感
        'water': re.compile('水'),
        'search': re.compile('[d|b]fs', re.I), #[]
        'sort': re.compile('排序'),
        'tree': re.compile('树'),
        'violence': re.compile('暴力')
    }
    tags = []
    for m in match:
        tags += re.findall(match[m], discuss_text)
    return tags

def write_to_file(item):
    with open('HDU题目类型.txt', 'a', encoding='utf-8') as f:
        f.write(json.dumps(item, ensure_ascii=False)+'\n')
def write_to_json(content):
    with open('HDU题目类型.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False)+'\n')
def write_to_csv(content): 
    with open('HDU题目类型.csv', 'a', encoding='utf-8-sig', newline='') as f:
        #newline=''取消自动换行
        #注意：utf-8还是乱码，得设置为utf-8-sig
        fieldnames = ['Pro_ID', 'Title', 'Tags', 'Ratio', 'Link']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(content)
        
def create_sql():
    db = pymysql.connect(host='localhost',user='root',password='123456',port=3306)
    cursor = db.cursor()
    cursor.execute("CREATE DATABASE spiders DEFAULT CHARACTER SET utf8") #创建数据库spiders
    
    db = pymysql.connect(host='localhost', user='root', password='123456', port=3306, db='spiders')
    cursor = db.cursor() 
    sql= "CREATE TABLE HDU (Pro_ID VARCHAR(255) NOT NULL,Title VARCHAR(255) NOT NULL,Tags VARCHAR(255) NOT NULL,Ratio VARCHAR(255) NOT NULL,Link VARCHAR(255) NOT NULL)"
    cursor.execute(sql) #执行SQL语句，创建数据表HDU
    db.close()
    
def write_to_sql(data):
    table = 'HDU' #表名
    keys = ', '.join(data.keys())
    values = ', '.join(['%s'] * len(data))
    sql = 'INSERT INTO {table}({keys}) VALUES ({values}) ON DUPLICATE KEY UPDATE '.format(table=table, keys=keys, values=values)
    update = ','.join(["{key} = %s".format(key=key) for key in data])
    sql += update 
    db = pymysql.connect(host='localhost', user='root', password='123456', port=3306, db='spiders')
    cursor = db.cursor()
    try:
        if cursor.execute(sql, tuple(data.values())*2):
            print('Successful')
            db.commit()
    except:
        print('Failed')
        db.rollback()
    db.close() 
    
def main():
    #Problem Archive页面经动态渲染，requests无法爬取，
    #于是利用selenum模拟浏览器操作，实现所见即所得
    browser = webdriver.Chrome()
    #create_sql() #第一次创建时使用
    for i in range(1,2):
        url = 'http://acm.hdu.edu.cn/listproblem.php?vol=' + str(i)
        browser.get(url)
        content = []
        text = browser.page_source
        print('正在爬取第', i, '页')
        for item in parse_Problem_Archive(text):
            print(item)
            write_to_file(item)
            write_to_sql(item)
            content.append(item)
        write_to_json(content)
        write_to_csv(content)
        time.sleep(3)
    browser.close()
        
main()

正在爬取第 1 页
{'Pro_ID': '1000', 'Title': 'A + B Problem', 'Tags': '水', 'Ratio': '29.85%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1000'}
Successful
{'Pro_ID': '1005', 'Title': 'Number Sequence', 'Tags': '水', 'Ratio': '25.38%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1005'}
Successful
{'Pro_ID': '1006', 'Title': 'Tick and Tick', 'Tags': '暴力', 'Ratio': '27.28%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1006'}
Successful
{'Pro_ID': '1007', 'Title': 'Quoit Design', 'Tags': '树,水', 'Ratio': '26.66%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1007'}
Successful
{'Pro_ID': '1008', 'Title': 'Elevator', 'Tags': '水', 'Ratio': '54.23%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1008'}
Successful
{'Pro_ID': '1009', 'Title': "FatMouse' Trade", 'Tags': '水', 'Ratio': '34.48%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1009'}
Successful
{'Pro_ID': '1010', 'Title': 'Tempter of the Bone', 'Tags': 'DFS,BFS', 'Ratio': '26.55%', 'Link': 'http://acm.hdu.

{'Pro_ID': '1073', 'Title': 'Online Judge', 'Tags': '水', 'Ratio': '36.63%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1073'}
Successful
{'Pro_ID': '1074', 'Title': 'Doing Homework', 'Tags': 'DP', 'Ratio': '48.45%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1074'}
Successful
{'Pro_ID': '1076', 'Title': 'An Easy Task', 'Tags': '水,暴力', 'Ratio': '64.35%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1076'}
Successful
{'Pro_ID': '1080', 'Title': 'Human Gene Functions', 'Tags': 'DP', 'Ratio': '54.92%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1080'}
Successful
{'Pro_ID': '1081', 'Title': 'To The Max', 'Tags': 'DP,水,dp,暴力', 'Ratio': '45.52%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1081'}
Successful
{'Pro_ID': '1082', 'Title': 'Matrix Chain Multiplication', 'Tags': '水', 'Ratio': '61.20%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1082'}
Successful
{'Pro_ID': '1084', 'Title': 'What Is Your Grade?', 'Tags': '水', 'Ratio': '31.91%', 'Link': 'h

In [8]:
import re
import csv
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from requests.exceptions import RequestException
browser = webdriver.Chrome()
url = 'http://acm.hdu.edu.cn/listproblem.php?vol=' + str(10)
browser.get(url)
text = browser.page_source

In [40]:
def parse_Problem_Archive(text):
    soup = BeautifulSoup(text, 'lxml')
    items = soup.find_all(align='center', height='22')
    for i in range(1, len(items)):
        Pro_ID = items[i].td.text #第一个td的文本内容
        tags = ','.join(parse_discuss_page(Pro_ID))
        if len(tags) == 0:
            continue
        data = {
            'Pro_ID': Pro_ID,
            'Title': items[i].a.text.strip(),
            'Tags': list(set(tags)), #利用set去重
            'Ratio': items[i].td.next_sibling.next_sibling.text[:6],
            'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=' + Pro_ID
        }
        yield data

In [29]:
import csv

def write_to_file(item):
    with open('HDU题目类型.txt', 'a', encoding='utf-8') as f:
        f.write(json.dumps(item, indent=2, ensure_ascii=False)+'\n')
def write_to_json(content):
    with open('HDU题目类型.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, indent=2, ensure_ascii=False)+'\n')
def write_to_csv(content): 
    with open('HDU题目类型.csv', 'a', encoding='utf-8-sig', newline='') as f:
        #newline=''取消自动换行
        #注意：utf-8还是乱码，得设置为utf-8-sig
        fieldnames = ['Pro_ID', 'Title', 'Tags', 'Ratio', 'Link']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(content)
        


In [41]:
content = []
items = parse_Problem_Archive(text)


In [42]:



for item in items:
    print(item)
    write_to_sql(item)
    #write_to_file(item)
    #content.append(item)
#write_to_json(content)
#write_to_csv(content)

{'Pro_ID': '1902', 'Title': 'The Dragon of Loowater', 'Tags': ['水'], 'Ratio': '39.28%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1902'}
Successful
{'Pro_ID': '1905', 'Title': 'Pseudoprime numbers', 'Tags': ['水', ','], 'Ratio': '47.53%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1905'}
Failed
{'Pro_ID': '1908', 'Title': 'Double Queue', 'Tags': ['水', ',', '树'], 'Ratio': '50.75%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1908'}
Failed
{'Pro_ID': '1930', 'Title': 'And Now, a Remainder from Our Sponsor', 'Tags': ['水', ','], 'Ratio': '46.42%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1930'}
Failed
{'Pro_ID': '1939', 'Title': 'He is offside!', 'Tags': ['水'], 'Ratio': '60.81%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1939'}
Successful
{'Pro_ID': '1963', 'Title': 'Investment', 'Tags': ['d', 'p'], 'Ratio': '43.53%', 'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=1963'}
Failed
{'Pro_ID': '1969', 'Title': 'Pie', 'Tags': ['水'], 'Ratio': '34.38%'

In [36]:
tags = set(['力', '序', '排', '暴力'])
tags = ','.join(tags)

In [17]:
tags

'暴力,力,序,排'

AttributeError: 'str' object has no attribute 'len'

In [173]:
import re
import time
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from requests.exceptions import RequestException

In [174]:
def parse_first_page(text):
    soup = BeautifulSoup(text, 'lxml')
    items = soup.find_all(align='center', height='22')
    for i in range(1, len(items)):
        Pro_ID = items[i].td.text #第一个td的文本内容
        tags = parse_discuss_page(Pro_ID)
        if !tags:
            continue
        data = {
            'Pro.ID': Pro_ID,
            'Title': items[i].a.text.strip(),
            'Tags': tags,
            'Ratio': items[i].td.next_sibling.next_sibling.text[:6],
            'Link': 'http://acm.hdu.edu.cn/showproblem.php?pid=' + Pro_ID
        }
        yield data

In [175]:
def get_one_page(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) '
            + 'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        return None
    except RequestException:
        return None

In [176]:
def parse_discuss_page(Pro_ID):
    url = 'http://acm.hdu.edu.cn/discuss/problem/list.php?problemid=' + str(Pro_ID)
    text = get_one_page(url)
    soup = BeautifulSoup(text, 'lxml')
    discuss_list = soup.select('#ulframe > li > a:nth-child(2)')
    discuss_text = ''
    for d in discuss_list:
        discuss_text += d.text
    match = {
        'dp': re.compile('dp', re.I), #re.I使匹配对大小写不敏感
        'water': re.compile('水'),
        'search': re.compile('(d|b)fs', re.I),
        'sort': re.compile('排序'),
        'tree': re.compile('树'),
        'violence': re.compile('暴力')
    }

    tags = []
    for m in match:
        tags += re.findall(match[m], discuss_text)
    return tags

In [172]:
def main():
    browser = webdriver.Chrome()
    for i in range(10,11)
        url = 'http://acm.hdu.edu.cn/listproblem.php?vol=' + str(i)
        browser.get(url)
        text = browser.page_source
        for item in parse_first_page(text):
            print(item)
        time.sleep(3)

['暴力']


In [177]:
main()

正在爬取第 1 页
{'image': '//img14.360buyimg.com/n7/jfs/t1/14141/3/11626/192739/5c908772E555959fd/35a455546a00318d.jpg', 'price': '3536.00', 'comment': '27万+', 'title': '【iPad年货节，爆款送爱奇艺会员，更有以旧换新至高补贴1000元】iPad新年优惠购，你值得拥有！前往iPad会场》》', 'shop': 'Apple产品京东自营旗舰店'}
{'image': '//img12.360buyimg.com/n7/jfs/t1/46679/34/10345/87214/5d780bd9E4de405a3/a603ba3453b980b4.jpg', 'price': '2799.00', 'comment': '10万+', 'title': '【iPad年货节，爆款送爱奇艺会员，更有以旧换新至高补贴1000元】iPad新年优惠购，你值得拥有！前往iPad会场》》', 'shop': 'Apple产品京东自营旗舰店'}
{'image': '//img13.360buyimg.com/n7/jfs/t1/110055/36/3596/159822/5e13d3f9Eaf314c3f/4eab5d76887b613c.jpg', 'price': '2299.00', 'comment': '9.3万+', 'title': '【今日钜惠】下单抽iPhone11Pro等豪礼，套餐版晒图再送蓝牙耳机！全国多仓速发·急速到达~购套餐够实惠~ipad礼盒装限量发售，点击抢购~', 'shop': 'APPLE翔合专卖店'}
{'image': '//img11.360buyimg.com/n7/jfs/t1/59393/17/10464/225685/5d7f63c5E5e23f76a/0589d32182617442.jpg', 'price': '4388.00', 'comment': '152万+', 'title': '苹果,Apple,苹果平板,平板电脑,iPad,苹果新品,新iPad,iPad新品,iPad2019年秋季新款,平板电脑,平板,pingban', 'shop': 'Apple产品京东自营旗

In [190]:
llll = []
llll += ['dfs']


In [191]:
llll

['dfs']

In [204]:
ts = []

In [211]:
ts+=(re.findall('[d|b]fs', 'dfs'))

In [212]:
ts

[['d'], ['d'], ['dfs'], 'dfs']